<a href="https://colab.research.google.com/github/dxareal/Driving_Data/blob/main/st_23_ca2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# We are using Github Copilot and ChatGPT

In [21]:
# Clone data from public repo
!git clone https://github.com/dxareal/Driving_Data.git

fatal: destination path 'Driving_Data' already exists and is not an empty directory.


In [22]:
import numpy as np
import os, cv2
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import time
import keras
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Convolution2D, Flatten, Dense, Dropout
import tensorflow as tf

In [23]:
DATA_BASE_PATH = 'Driving_Data'

# Track paths always have 'forward', 'backward' and 'lane_correction' folders
TRACK_1_PATH = os.path.join(DATA_BASE_PATH, 'Track_1')
TRACK_2_CENTER_PATH = os.path.join(DATA_BASE_PATH, 'Track_2/center')
TRACK_2_LEFT_PATH = os.path.join(DATA_BASE_PATH, 'Track_2/left_lane')

DATA_FOLDERS = ['forward', 'backward', 'lane_correction']

LOG_FILE = 'driving_log.csv'
IMG_FOLDER = 'IMG'

COLUMNS = ['center', 'left', 'right', 'steering', 'throttle', 'reverse', 'speed']

In [24]:
def main():
    track_1_data = process_track_data(TRACK_1_PATH)
    #track_2_data = process_track_data(TRACK_2_CENTER_PATH)

    print(track_1_data.head())
    print(track_1_data.shape)

    # print(track_2_data.head())
    # print(track_2_data.shape)

In [25]:
# https://arxiv.org/pdf/1604.07316v1.pdf
def nvidia_model():
    model = Sequential()
    model.add(Convolution2D(24, kernel_size=(5,5), strides=(2,2), input_shape=(66, 200, 3), activation='elu'))
    model.add(Convolution2D(36, kernel_size=(5,5), strides=(2,2), activation='elu'))
    model.add(Convolution2D(48, kernel_size=(5,5), strides=(2,2), activation='elu'))
    model.add(Convolution2D(64, kernel_size=(3,3), activation='elu'))
    model.add(Convolution2D(64, kernel_size=(3,3), activation='elu'))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(100, activation = 'elu'))
    model.add(Dropout(0.5))
    model.add(Dense(50, activation = 'elu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation = 'elu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))

    optimizer = Adam(learning_rate = 0.001)
    model.compile(loss='mse', optimizer = optimizer, metrics=['accuracy'])
    return model

In [26]:
def img_preprocess(img):
    img = mpimg.imread(img)
    img = img[60:135, :, :]
    img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    img = cv2.GaussianBlur(img, (3,3), 0)
    img = cv2.resize(img, (200, 66))
    img = img/255
    return img

In [27]:
def path_leaf(path):
    head, tail = os.path.split(path)
    return tail

def adjust_path(image_path, track_path, folder):
    return os.path.join(track_path, folder, IMG_FOLDER, path_leaf(image_path))

In [28]:
def process_track_data(track_path):
    data = pd.DataFrame()  # Initialize an empty DataFrame
    for folder in DATA_FOLDERS:
        data_path = os.path.join(track_path, folder, LOG_FILE)
        folder_data = pd.read_csv(data_path, names=COLUMNS)
        folder_data['center'] = folder_data['center'].apply(adjust_path, track_path=track_path, folder=folder)
        folder_data['left'] = folder_data['left'].apply(adjust_path, track_path=track_path, folder=folder)
        folder_data['right'] = folder_data['right'].apply(adjust_path, track_path=track_path, folder=folder)
        data = pd.concat([data, folder_data])  # Append the data from the current folder#
        data.reset_index(drop=True, inplace=True)  # Reset the index of the combined DataFrame
    return data

In [29]:
main()

                                              center  \
0  Driving_Data/Track_1/forward/IMG/D:\auto_files...   
1  Driving_Data/Track_1/forward/IMG/D:\auto_files...   
2  Driving_Data/Track_1/forward/IMG/D:\auto_files...   
3  Driving_Data/Track_1/forward/IMG/D:\auto_files...   
4  Driving_Data/Track_1/forward/IMG/D:\auto_files...   

                                                left  \
0  Driving_Data/Track_1/forward/IMG/D:\auto_files...   
1  Driving_Data/Track_1/forward/IMG/D:\auto_files...   
2  Driving_Data/Track_1/forward/IMG/D:\auto_files...   
3  Driving_Data/Track_1/forward/IMG/D:\auto_files...   
4  Driving_Data/Track_1/forward/IMG/D:\auto_files...   

                                               right  steering  throttle  \
0  Driving_Data/Track_1/forward/IMG/D:\auto_files...       0.0       0.0   
1  Driving_Data/Track_1/forward/IMG/D:\auto_files...       0.0       0.0   
2  Driving_Data/Track_1/forward/IMG/D:\auto_files...       0.0       0.0   
3  Driving_Data/Track_